<a href="https://colab.research.google.com/github/silentfortin/ai-portfolio/blob/main/01-contactease-python/ContactEase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ContactEase – Contact Manager in Python
👨‍💻 Project developed during the AI Engineering Master – Week 1

🔗 GitHub Repo: [ai-portfolio](https://github.com/silentfortin/ai-portfolio/)


In [5]:
!pip install questionary

In [66]:
import sys
import time
import uuid
import json
import questionary


from json.decoder import JSONDecodeError

In [67]:
# ANSI color codes
RED = "\033[91m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
CYAN = "\033[96m"
RESET = "\033[0m"

In [97]:
# Class representing a User with name and surname
class User:
  # Constructor to initialize name and surname
  def __init__(self, name, surname):
    self.name = name
    self.surname = surname

class Contact:
    def __init__(self, name, surname, phone_number):
        self.id = str(uuid.uuid4())
        self.name = name
        self.surname = surname
        self.phone = phone_number

    def __str__(self):
        return f'{GREEN}Full name: {self.name} {self.surname} \| Phone: {self.phone}{RESET}'

class ContactsBook:

  def __init__(self):
    self.contacts = []

  def add_contact(self, contact):
      self.contacts.append(contact)
      self.save_to_file()
      print(f'{GREEN}{contact.name} {contact.surname} has been added.\n')


  def show_all_contacts(self):
    if len(self.contacts) != 0:
      print(f'\n{CYAN}Your contacts: {RESET}')
      for contact in self.contacts:
          print(f"{contact.name} {contact.surname} | Tel: {contact.phone}")
    else:
      print(f'{YELLOW} Your contacts book is empty.\n{RESET}')

  def edit_contact(self, contact, mod, updated_var):
      """
      User will insert 'mod' which is the element that wants to modify and then
      the new value 'updated_var'. When the case match, the var is modified and
      the file pudated
      """
      match mod:
        case 'n':
          contact.name = updated_var
          print(f'{GREEN}Contact name has been modified.\n')
        case 's':
          contact.surname = updated_var
          print(f'{GREEN}Contact surname has been modified.\n')
        case 'p':
          contact.phone = updated_var
          print(f'{GREEN}Contact phone number has been modified.\n')
        case _:
            print(f'{RED}ERROR: {contact.name} {contact.surname} has not been modified.\n')
            return

      self.save_to_file()

  def remove_contact(self, contact_to_remove):
      for contact in self.contacts:
          if contact.id == contact_to_remove.id:
              self.contacts.remove(contact)
              # saving
              self.save_to_file()
              print(f"{YELLOW}Contact {contact.name} {contact.surname} removed.{RESET}")
              return
      print("Contact not found.")

  def search_contact(self, name, surname):
    name = name.strip().lower()
    surname = surname.strip().lower()

    for contact in self.contacts:
      if contact.name.lower() == name and contact.surname.lower() == surname:
        return contact
    return None

  def save_to_file(self, filename="contacts_book.json"):
      try:
        with open(filename, "w") as f:
          # Convert each Contact object to a dictionary
          data = [vars(c) for c in self.contacts]
            # Save the list of contacts as JSON with indentation
          json.dump(data, f, indent=2)
      except JSONDecodeError as e:
          print(f'Error: Failed to encode contacts to JSON. Details: {e}')
      except Exception as e:
          print(f'Unexpected error while saving contacts: {e}')

  def load_from_file(self, filename="contacts_book.json"):
      try:
        with open(filename, "r") as file:
          # Load list of dictionaries from JSON
          data = json.load(file)
          # Recreate Contact objects from dictionaries (that's why I'm using **c)
          self.contacts = [Contact(**c) for c in data]
      except FileNotFoundError:
          print(f'Warning: File "{filename}" not found. Starting with an empty contact list.')
          self.contacts = []
      except JSONDecodeError as e:
          print(f'Warning: File contains invalid JSON. Details: {e}')
          self.contacts = []
      except Exception as e:
          print(f'Unexpected error while loading contacts: {e}')
          self.contacts = []


In [98]:
# helper function(s)
def edit_contact_helper(book):
    possible_choices = ['n', 'p', 's']

    print(f'{YELLOW}Insert the name and the surname of the contact that you want to edit.{RESET}')
    name = input('\nInsert contact name: ').strip()
    surname = input('Insert contact surname: ').strip()
    contact_to_edit = book.search_contact(name, surname)

    if not contact_to_edit:
        print(f'{RED}Contact not found.{RESET}')
        return

    while True:
        edit_choice = input(
            '\nPress [N] to edit the name\nPress [S] to edit the surname\nPress [P] to edit the phone number\n'
        ).strip().lower()
        if edit_choice in possible_choices:
            break
        else:
            print(f'{RED}Invalid value. Please enter N, S, or P.{RESET}')

    new_value = input('Insert the new value: ').strip()
    book.edit_contact(contact_to_edit, edit_choice, new_value)


### CLI

In [100]:
book = ContactsBook()
# book.load_from_file()  # Load or create new file if not found

def main_menu():
    while True:
        print(f'\n{CYAN}=== Contact Book ==={RESET}')
        print(f'{GREEN}1. Add contact{RESET}')
        print(f'{GREEN}2. Show contacts{RESET}')
        print(f'{GREEN}3. Modify contact{RESET}')
        print(f'{GREEN}4. Remove contact{RESET}')
        print(f'{GREEN}5. Search contact{RESET}')
        print(f'{RED}6. Exit{RESET}')

        choice = input(f'Choose an option (1-6): ')

        if choice == '1':
            name = input('\nInsert contact name: ')
            surname = input('Insert contact surname: ')
            phone = input('Insert phone number: ')

            new_contact = Contact(name, surname, phone)
            # checking if contact already exists before adding
            check_duplicate = book.search_contact(new_contact.name, new_contact.surname)

            if check_duplicate == None:
              added = book.add_contact(new_contact)
            else:
              print(f'{YELLOW}The contact {new_contact.name} {new_contact.surname} already exists. Do you want to add it anyways?{RESET}')
              add_choice = input(f'Select [Y] or [X]: ')
              if add_choice.lower() == 'y':
                added = book.add_contact(new_contact)
              else:
                print(f'{YELLOW}The contact has not been added.\n{RESET}')

        elif choice == '2':
            book.show_all_contacts()

        elif choice == '3':
          edit_contact_helper(book)

        elif choice == '4':
          name = input('\nInsert contact name: ')
          surname = input('Insert contact surname: ')

          contact_to_remove = book.search_contact(name, surname)
          book.remove_contact(contact_to_remove)

        elif choice == '5':
          name = input('\nInsert contact name: ')
          surname = input('Insert contact surname: ')

          contact = book.search_contact(name, surname)
          if contact:
              print(contact)
          else:
              print(f'{YELLOW}Contact not found.{RESET}')

        elif choice == '6':
            print(f'{RED}Exiting...{RESET}')
            break

        else:
            print(f'{RED}Invalid choice. Try again.{RESET}')


In [96]:
if __name__ == "__main__":
    main_menu()


=== Contact Book ===
1. Add contact
2. Show contacts
3. Modify contact
4. Remove contact
5. Search contact
6. Exit
Choose an option (1-6): 1

Insert contact name: a
Insert contact surname: b
Insert phone number: 312
a b has been added.


=== Contact Book ===
1. Add contact
2. Show contacts
3. Modify contact
4. Remove contact
5. Search contact
6. Exit
Choose an option (1-6): 3
Insert the name and the surname of the contact that you want to edit.

Insert contact name: a
Insert contact surname: b

Press [N] to edit the name
Press [S] to edit the surnamename
Press [P] to edit the phone number
j
Invalid value. Please try again.


In [12]:
help(questionary.text)

Help on function text in module questionary.prompts.text:

text(message: str, default: str = '', validate: Any = None, qmark: str = '?', style: Optional[prompt_toolkit.styles.style.Style] = None, multiline: bool = False, instruction: Optional[str] = None, lexer: Optional[prompt_toolkit.lexers.base.Lexer] = None, **kwargs: Any) -> questionary.question.Question
    Prompt the user to enter a free text message.
    
    This question type can be used to prompt the user for some text input.
    
    Example:
        >>> import questionary
        >>> questionary.text("What's your first name?").ask()
        ? What's your first name? Tom
        'Tom'
    
    .. image:: ../images/text.gif
    
    This is just a really basic example, the prompt can be customised using the
    parameters.
    
    Args:
        message: Question text.
    
        default: Default value will be returned if the user just hits
                 enter.
    
        validate: Require the entered value to pass a 